In [ ]:
import numpy as np
import pandas as pd
import pandas.io.sql as sql

import os
import joblib
import mysql.connector

from dotenv import load_dotenv
from sqlalchemy import create_engine

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [ ]:
def connect_cloud():
    load_dotenv()

    config = {
        "host": os.getenv("AIVEN_DB_HOST"),
        "port": int(os.getenv("AIVEN_DB_PORT")),
        "user": os.getenv("AIVEN_DB_USER"),
        "password": os.getenv("AIVEN_DB_PASS"),
        "database": os.getenv("AIVEN_DB_NAME"),
        "ssl_ca": os.getenv("AIVEN_DB_CA")
    }

    try:
        conn = mysql.connector.connect(**config)
        print("Connected to cloud database.")
        return conn
    except Exception as e:
        print(f"Connection error: {e}")

def create_database(cursor):
    try:
        cursor.execute("CREATE DATABASE IF NOT EXISTS PROJ_ADY")

    except:
        return "Create database error."

def create_table(cursor):
    try:
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS tbl_motobike(
            ID INT AUTO_INCREMENT PRIMARY KEY,
            Price VARCHAR(50),
            Min_Price VARCHAR(50),
            Brand VARCHAR(50),
            Model VARCHAR(100),
            Year_of_Manufacture VARCHAR(50),
            Kilometer_Driven VARCHAR(50),
            Location VARCHAR(255),
            Listing_Time VARCHAR(50)
        );
        '''
        cursor.execute(create_table_query)
        print("Table created or already exists.")
    except Exception as e:
        print(f"Create table error: {e}")

def insert_data(cursor, df):
    try:
        insert_query = '''
        INSERT INTO tbl_motobike(
            Price, 
            Min_Price, 
            Brand, 
            Model, 
            Year_of_Manufacture, 
            Kilometer_Driven, 
            Location, 
            Listing_Time
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        '''
        columns = ['Price', 'Min_Price', 'Brand', 'Model', 'Year_of_Manufacture', 'Kilometer_Driven', 'Location', 'Listing_Time']
        cursor.executemany(insert_query, df[columns].values.tolist())
        print("Data inserted successfully.")
    except Exception as e:
        print(f"Insert error: {e}")

In [ ]:
def data_preprocessing(df):
    df['name'] = df['Brand'] + ' ' + df['Model']
    df["name"], name_labels = pd.factorize(df["name"])
    joblib.dump(name_labels, 'name_labels.pkl')

    df['Price'] = df['Price'].apply(lambda x: int(x.replace('.', '').replace(' đ', '').replace('đ', '').strip()))
    df['Showroom_Price'] = pd.to_numeric(df['Min_Price'].str.lower().str.replace('triệu', '').str.strip(), errors='coerce') * 1000000

    df['Kilometer Driven'] = df['Kilometer Driven'].replace('Không có thông tin', np.nan)
    df['Kilometer Driven'] = pd.to_numeric(df['Kilometer Driven'], errors='coerce')
    df['Year of Manufacture'] = df['Year of Manufacture'].str.extract(r'(\d{4})')
    df['Year of Manufacture'] = pd.to_numeric(df['Year of Manufacture'], errors='coerce')
    for col in df.select_dtypes(include=[np.number]).columns:
        df[col] = df[col].fillna(df[col].mean())

    df = df.drop(['Min_Price', 'Brand', 'Model', 'Địa điểm', 'Thời gian đăng'], axis=1)

    # print(df.head())

def training_model(df):
    X = df.drop(columns=['Price'])
    y = df['Price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    # model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f'MSE: {mse}')
    print(f'R2 Score: {r2}')

    joblib.dump(model, 'bike_price_model.pkl')

In [ ]:
def preprocess_input(user_input):
    name_labels = joblib.load('name_labels.pkl')
    
    df_input = pd.DataFrame([user_input])

    df_input['name'] = name_labels.get_indexer([df_input['name'][0]])[0]
    df_input['Year of Manufacture'] = pd.to_numeric(df_input['Year of Manufacture'], errors='coerce')
    df_input['Kilometer Driven'] = pd.to_numeric(df_input['Kilometer Driven'], errors='coerce')

    df_input = df_input.fillna(0)

    return df_input

def predict_price(data_input):
    processed_input = preprocess_input(data_input)
    
    model = joblib.load('bike_price_model.pkl')
    prediction = model.predict(processed_input)
    
    return prediction[0]

In [ ]:
def main():
    file_path = "E:/Python/3SP25/CHOTOT_motorcycles.csv"
    df = pd.read_csv(file_path)
    
    conn = connect_cloud()
    try:
        cursor = conn.cursor()
        
        create_database(cursor)
        conn.database = "PROJ_ADY"
        
        create_table(cursor)
        
        df.rename(columns={
            'Year of Manufacture': 'Year_of_Manufacture',
            'Kilometer Driven': 'Kilometer_Driven',
            'Địa điểm': 'Location',
            'Thời gian đăng': 'Listing_Time'
        }, inplace=True)
        
        insert_data(cursor, df)
        
        print(df.head())
        
        data_preprocessing(df)
        training_model(df)
        
        conn.commit()
    except:
        print('Main error!')
    finally:
        cursor.close()
        conn.close()
        print("Connection closed.")

    data_input = {
        'Year of Manufacture': '2016',
        'Kilometer Driven': '21100',
        'name': 'Honda Future',
        'Showroom_Price': '40000000',
    }

    price = predict_price(data_input)
    print(f"Giá xe dự đoán: {price:,.0f} VNĐ")

if __name__ == "__main__":
    main()